### Sharded & Scattered Retrieval
As a knowledge base grows from thousands to millions or billions of documents …

A single, monolithic vector store becomes a major bottleneck. Search latency increases, and the index becomes unwieldy to manage and update.

<p align="center">
  <img src="../../figures/shard_retrieval.png" width="800">
</p>

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True